# dataset

> kaiming normalization for attention layers

In [ ]:
#| default_exp dataset

In [ ]:
#| hide
from nbdev.showdoc import *
# import gatcadnet
# import GatCadNet_refactored
#from gatcadnet.core import *
from GatCadNet_refactored.core import *

In [ ]:
path

Path('.')

In [ ]:
class floorPlanCad_dataset(Dataset):
    
    def __init__(self, root,transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        #print(f'init called')
        
        self.root = root
        super().__init__(root, transform, pre_transform)
        # print(self.processed_dir)
        # print(self.raw_dir)
        # print(self.raw_paths)
        # print(type(Path(self.processed_dir)))
        
    @property
    def raw_file_names(self):
        #print(f'raw_file_names called')
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        self.filenames = next(walk(self.root/"raw"), (None, None, []))[2]
        return self.filenames

    @property
    def processed_file_names(self):
        #print(f'processed_file_names called')
        """ If these files are found in raw_dir, processing is skipped"""

        #return [f'd_{self.filenames.index(i)}.pt' for i in self.filenames]
        return [f'd_{i[:-4]}.pt' for i in self.filenames]
    

    def download(self):
        pass
 
    def process(self):  
        # print(f'process called')
        # edge_indices = [[1, 2], [2, 1]]
        # edge_torch = torch.tensor(edge_indices)
        # edge_torch = edge_torch.t().to(torch.long).view(2, -1)
        
        for i in tqdm(self.filenames): 
            
            #Node Features
            #------------------------------------------------------------------------
            current_drw = Path(self.raw_dir)/i
            drw_paths, drw_attributes = svg2paths(current_drw)
            node_features = []
            #node_features = torch.tensor([], dtype=torch.float)
            legal_paths=[]
            legal_attributes=[]
            node_class =[]
            removed_files = []
            # remove corrupt data
        
            for drw_attribute in drw_attributes:
                if ('semantic-id' and 'instance-id') in drw_attribute:
                    path_index = drw_attributes.index(drw_attribute)
                    drw_path = drw_paths[path_index]
                    node_class.append(int(drw_attribute['semantic-id']))
                    legal_paths.append(drw_path)
                    legal_attributes.append(drw_attribute)
            #print(len(legal_paths),len(legal_attributes))
            #some drawing has no legal paths        
            if not legal_paths:
                removed_files.append(i)
                (Path(self.raw_dir)/i).unlink()
                continue
            for legal_path in legal_paths:
                node_feature = self.node_features(legal_path)
                node_features += node_feature
                

            node_features = torch.tensor(node_features, dtype=torch.float)
            node_features = node_features.view(node_features.shape[0]//4,-1)
            node_class = torch.tensor(node_class)
            
            #y = torch.randint(0, 2, (node_features.shape[0],))
            
            #-------------------------------------------------------------------------
            #Edge Index
            edge_index =[]
            edge_feats =[]
            node_instance_matrix = []
            for legal_path in legal_paths:
                to_others=[]
                for other_legal_path in legal_paths:
                    if  (self.positional_offset(legal_path,other_legal_path)[1] < 4) or (self.start_to_end(legal_path,other_legal_path)):
                        # Edge Index
                        idx1 = legal_paths.index(legal_path)
                        idx2 = legal_paths.index(other_legal_path)
                        edge_index.append([idx1,idx2])
                        #------------------------------------------------------------------------- 
                        # Edge Features
                        
                        edge_feats.append(self.edge_features(legal_path,other_legal_path))
                        #this can be used for above loop too. Normally it creates self loops.
                        same = (idx1 == idx2)
                        
                        if (not same):                            
                            if (legal_attributes[idx1]['instance-id'] == legal_attributes[idx2]['instance-id']):                            
                                node_instance_matrix.append(1)
                            else:
                                node_instance_matrix.append(0)
                            
                        
                    #     #------------------------------------------------------------------------- 
                    # if (legal_path['instance-id']==other_legal_path['instance-id']):
                    #     print('same')
            # test value for extra property eventually. i reserved this for the second 'y' value 
            # for graph level prediction (instance prediction)
#            node_instance_matrix = []
#this is the case for that every node is included.            
#             for legal_attribute in legal_attributes:                
#                 for other_legal_attribute in legal_attributes:
#                     if  legal_attribute['instance-id'] == other_legal_attribute['instance-id']:
#                         print(legal_attribute['instance-id'],other_legal_attribute['instance-id'])
#                         node_instance_matrix.append(1)
#                     else:
#                         node_instance_matrix.append(0)
                        
                    
            
  
            edge_index = torch.tensor(edge_index)
            edge_index = edge_index.t().to(torch.long).view(2, -1)
            #print(edge_index.shape)
            edge_feats = torch.tensor(edge_feats, dtype=torch.float)
            #print(len(node_instance_matrix))
            node_instance_matrix = torch.tensor(node_instance_matrix, dtype=torch.float).unsqueeze(1)
            #print(node_instance_matrix.shape)
            node_instance_matrix = torch.abs(torch.cat((node_instance_matrix,node_instance_matrix-1),dim=1).to(torch.float))
            data = Data(x=node_features, 
                                edge_index = edge_index,
                                #edge_attr = torch.tensor([2,2], dtype=torch.float),
                                edge_attr = edge_feats,
                                y=node_class, yy=node_instance_matrix)
            
            data.edge_index,data.edge_attr = remove_self_loops(data.edge_index,data.edge_attr)
            
            torch.save(data,os.path.join(self.processed_dir,f'd_{i[:-4]}.pt'))
        print(removed_files)

    def path_to_seg(self,d_path):
        return d_path[0]

    def angle_line(self,d_path):
        angle_max=6.28
        seg = self.path_to_seg(d_path)
        delta = seg.point(1) - seg.point(0)
        polar = cmath.polar(delta)
        angle = polar[-1]
        if angle < 0 : angle += 2* pi
        #normalized angle
        return angle / angle_max

    def length(self,d_path):
        max_length = 66
        seg = self.path_to_seg(d_path)
        return seg.length() / max_length

    
    
    def path_type(self,d_path):
        prim_types= 'Line', 'Arc'
        seg = self.path_to_seg(d_path)
        one_hot=[]
        for d in prim_types:
            one_hot.append(d == seg.__class__.__name__)
        return one_hot


    def node_features(self,d_path):
        return [self.angle_line(d_path),self.length(d_path), self.path_type(d_path)[0],self.path_type(d_path)[1]]    
    
    #Edge features-------------------------------------
    def start_to_end(self,d_path1,d_path2):
        seg_1 = self.path_to_seg(d_path1)
        seg_2 = self.path_to_seg(d_path2)
        if self.is_close(seg_1.point(0),seg_2.point(0)) or self.is_close(seg_1.point(0),seg_2.point(1)) or self.is_close(seg_1.point(1),seg_2.point(0)) or self.is_close(seg_1.point(1),seg_2.point(1)):
            return True
        else:
            return False
    
    def a_difference(self,angle1,angle2):
        #this is for normalization
        angle_max = 1.5707963268
        #angle must be in radians
        #https://www.nagwa.com/en/explainers/407162748438/
        #90 degrees must be dealt because if m1 * m2 = -1 it gives error.
        m1 = tan(angle1)
        m2 = tan(angle2)
        if self.is_close(1,-(m1*m2)):
            return 1.5707963268 / angle_max
        else:
            tan_alpha = abs((m1-m2)/(1+m1*m2))        
            return (atan(tan_alpha)) / angle_max
    
    def is_close(self,first,second):
        return abs(first - second) < 1e-3
    
    def is_near_90(self,angle):    
        return abs(abs(angle)- pi/2) < 1e-3
    
    def is_near_180(self,angle):
        self.difference= abs(abs(angle)- pi)    
        return (self.difference <= 1e-3) or (abs(self.difference - pi) <= 1e-3)
    
    def positional_offset(self,d_path1,d_path2):
        max_ofset= 70
        seg_1 = self.path_to_seg(d_path1)
        seg_2 = self.path_to_seg(d_path2)
        normalized=(Line(seg_1.point(.5),seg_2.point(.5)).length()) / max_ofset
        offset =(Line(seg_1.point(.5),seg_2.point(.5)).length())
        return normalized,offset
    
    def length_ratio(self,d_path1,d_path2):
        seg_1 = self.path_to_seg(d_path1)
        seg_2 = self.path_to_seg(d_path2)
        return seg_1.length()/(seg_1.length()+seg_2.length())
    
    def is_parallel(self,d_path1,d_path2):

        if self.is_near_180((self.angle_line(d_path1)-self.angle_line(d_path2))):
            return True
        else:
            return False
    
    def is_orthogonal(self,d_path1,d_path2):
        return self.is_near_90((self.angle_line(d_path1)- self.angle_line(d_path2)))
    
    def edge_features(self,d_path1,d_path2):
        angle1 = self.angle_line(d_path1)
        angle2 = self.angle_line(d_path2)
        return [self.positional_offset(d_path1,d_path2)[0],self.length_ratio(d_path1,d_path2),self.is_parallel(d_path1,d_path2),self.is_orthogonal(d_path1,d_path2),self.start_to_end(d_path1,d_path2),self.a_difference(angle1,angle2)]


    #-------------------------------------------------- 
    
    def is_close(self,first,second):
        return abs(first - second) < 1e-3  
    
    def len(self):
        #print(f'len called')
        return len(self.filenames)

    def get(self, idx):
        # print(f'get called')
        data = torch.load(os.path.join(self.processed_dir,f'd_{self.filenames[idx][:-4]}.pt'))   
        return data   